# RAGを使ってModelerスクリプトを生成①コレクションの作成

## 必要なライブラリのインストール

!py -3.11 -m pip install ibm_watsonx_ai
!py -3.11 -m pip install langchain
!py -3.11 -m pip install langchain_milvus
!py -3.11 -m pip install langchain_community
!py -3.11 -m pip install pypdf
!py -3.11 -m pip install langchain_huggingface
!py -3.11 -m pip install langchain-ibm

## Milvusへの接続情報をセット

In [1]:
# Milvusへの接続情報

wxdata_host='xxxxxxxxxxxxxxxxxxxxxxx.lakehouse.appdomain.cloud' 
wxdata_port='xxxxx'
wxdata_user='ibmlhapikey'
wxdata_apikey='xxxxxxxxxxxxxxxxxxxxxxx'

#DB名
db_name ="default"
#コレクション名
collection_name = "spss_pdf"

#辞書形式のMilvus接続引数
milvus_connection_args ={
 'db_name':db_name,
 'uri': "https://"+wxdata_host+":"+wxdata_port,
 'user':wxdata_user,
 'password':wxdata_apikey,
 'secure':True
}



## embeddingsの指定

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

c:\Users\503615760\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## PDFの取得
PDFをダウンロードしてください。
https://www.ibm.com/docs/ja/SS3RA7_18.5.0/nl/ja/pdf/ModelerScriptingAutomation.pdf

#!wget https://www.ibm.com/docs/ja/SS3RA7_18.5.0/nl/ja/pdf/ModelerScriptingAutomation.pdf
!Invoke-WebRequest -Uri "https://www.ibm.com/docs/ja/SS3RA7_18.5.0/nl/ja/pdf/ModelerScriptingAutomation.pdf" -OutFile "ModelerScriptingAutomation.pdf"

## PDFの読み込み

In [4]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./ModelerScriptingAutomation.pdf") #ダウンロードしたPDFを指定
# PDF ドキュメントの内容をページで分割する
pages = loader.load_and_split() 

## Milvusにコレクションを作成し、ページのデータをInsert

一度に大量データを読み込むと以下のエラーになった<br>
RPC error: [batch_insert], <MilvusException: (code=65535, message=Broker: Message size too large: Please try with a smaller batch size.)>, <Time:{'RPC start': '2024-09-13 17:25:30.070110', 'RPC error': '2024-09-13 17:25:32.530650'}>
Failed to insert batch starting at entity: 0/500

In [5]:
#from langchain_community.vectorstores import Milvus
from langchain_milvus import Milvus

# 一度に大量データを読み込むとエラーになるのでn件ごとにINSERTする。
n=10

vector_store = Milvus(
    embeddings,
    connection_args=milvus_connection_args,
    drop_old=True, #既存コレクションは削除
    auto_id=True,
    collection_name = collection_name
)


for i in range(0,len(pages),n):
    vector_store.add_documents(documents=pages[i: i+n])

## 類似テキスト検索テスト

In [6]:
query="""可変長ファイルノードで"$CLEO_DEMOS/DRUG1n"を読み込み、ヒストグラムノードをつないで、Age列のヒストグラムを表示するModelerスクリプトをpythonで書いてください。""" 
docs = vector_store.similarity_search(query,k=5)

for doc in docs:
    print({"content": doc.page_content[0:1000], "metadata": doc.metadata} )

{'content': '表 26. ノードおよび出力\nノード名 出力名 コンテナ ID\ntable table "table"\nスクリプトの例\nstream = modeler.script.stream()\nfrom modeler.api import StorageType\n# Set up the variable file import node\nvarfilenode = stream.createAt("variablefile", "DRUG Data", 96, 96)\nvarfilenode.setPropertyValue("full_filename", "$CLEO_DEMOS/DRUG1n")\n# Next create the aggregate node and connect it to the variable file node\naggregatenode = stream.createAt("aggregate", "Aggregate", 192, 96)\nstream.link(varfilenode, aggregatenode)\n# Configure the aggregate node\naggregatenode.setPropertyValue("keys", ["Drug"])\naggregatenode.setKeyedPropertyValue("aggregates", "Age", ["Min", "Max"])\naggregatenode.setKeyedPropertyValue("aggregates", "Na", ["Mean", "SDev"])\n# Then create the table output node and connect it to the aggregate node\ntablenode = stream.createAt("table", "Table", 288, 96)\nstream.link(aggregatenode, tablenode)\n# Execute the table node and capture the resulting table output object\nresults = []\ntable

## バージョン情報

In [7]:
import sys
print('python:%s'% sys.version)

import ibm_watsonx_ai
print('ibm_watsonx_ai:%s'% ibm_watsonx_ai. __version__ )

import langchain
print('langchain:%s'% langchain. __version__ )

import langchain_community
print('langchain_community:%s'% langchain_community. __version__ )

import pypdf
print('pypdf:%s'% pypdf. __version__ )


print('langchain_huggingface:0.1.2')

print('langchain_milvus:0.1.7')

print('langchain-ibm:0.3.5')


python:3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
ibm_watsonx_ai:1.1.24
langchain:0.3.9
langchain_community:0.3.9
pypdf:5.1.0
langchain_huggingface:0.1.2
langchain_milvus:0.1.7
langchain-ibm:0.3.5
